In [2]:
from google.colab import files
uploaded = files.upload()

Saving train .csv to train .csv


Saving test.csv to test.csv


In [8]:
import numpy as np
import pandas as pd

X = pd.read_csv('train.csv')
X = X.drop(['Id'] , axis=1)
test_x0 = pd.read_csv('test.csv')
test_x = test_x0.drop('Id' , axis = 1)
y = X.pop('SalePrice')

cat_cols = [f for f in X.columns if X[f].dtype in ['object' , 'str']] #43 features
num_cols = [f for f in X.columns if X[f].dtype in ['int64','float64']] #37 features
cols = cat_cols + num_cols

test_x.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


In [11]:
imputer1 = SimpleImputer(strategy = 'mean')

X[num_cols] = imputer1.fit_transform(X[num_cols])
test_x[num_cols] = imputer1.transform(test_x[num_cols])

imputer2 = SimpleImputer(strategy = 'most_frequent')

X[cat_cols] = imputer2.fit_transform(X[cat_cols])
test_x[cat_cols] = imputer2.transform(test_x[cat_cols])


encoder = OneHotEncoder(handle_unknown = 'ignore')

X_oh = pd.DataFrame(encoder.fit_transform(X[cat_cols]).toarray())
X = pd.concat([X[num_cols],X_oh ], axis=1)
test_x_oh = pd.DataFrame(encoder.transform(test_x[cat_cols]).toarray())
test_x = pd.concat([test_x[num_cols],test_x_oh], axis=1)

#X = pd.get_dummies(X , columns = cat_cols)
#test_x = pd.get_dummies(test_x , columns = cat_cols)

scaler = StandardScaler()
X.columns = X.columns.astype(str)
test_x.columns =test_x.columns.astype(str)
X = pd.DataFrame(scaler.fit_transform(X))
test_x = pd.DataFrame(scaler.transform(test_x))


#print(X.isnull().any(),test_x.isnull().any())


In [54]:
from sklearn.model_selection import train_test_split

train_x , val_x , train_y , val_y = train_test_split(X , y , train_size = 0.8)

model = XGBRFRegressor(learning_rate = 1.0 ,
                       n_estimators = 350 ,
                       gamma = 0.2 ,
                       max_depth = 8,
                       reg_alpha = 1.0,
                       reg_delta = 0,
                       subsample = 0.8,
                       colsample_bynode = 0.8,
                       colsample_bytree = 0.8,
                       eval_metric = 'rmse')

'''grid = [{
    'learning_rate':[1.0],
    'n_estimators' : [200,300,400],
    'eval_metric':['rmse'],
    'gamma' : [0.1],
    'max_depth':[8],
    'reg_alpha':[0,0.1,1],
    'reg_lambda':[0,0.1,1],
    'objective':['reg:squarederror'],
    'subsample':[0.8],
    'colsample_bynode':[0.8],
    'colsample_bytree':[0.8]
}]'''

#print(help(model))

"grid = [{\n    'learning_rate':[1.0],\n    'n_estimators' : [200,300,400],\n    'eval_metric':['rmse'],\n    'gamma' : [0.1],\n    'max_depth':[8],\n    'reg_alpha':[0,0.1,1],\n    'reg_lambda':[0,0.1,1],\n    'objective':['reg:squarederror'],\n    'subsample':[0.8],\n    'colsample_bynode':[0.8],\n    'colsample_bytree':[0.8]\n}]"

In [55]:
'''GS = GridSearchCV(estimator = model ,
                  param_grid = grid,
                  cv = 3,
                  verbose = 4,
                  refit = True)'''

model.fit(train_x , train_y)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [09:55:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "reg_delta" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=0.8, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric='rmse', feature_types=None, gamma=0.2,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=8, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=350, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=1.0, ...)

Lasso(alpha=5)

In [56]:
from sklearn.metrics import r2_score
print(r2_score(val_y,model.predict(val_x)))

0.8646221572056842


In [57]:
predictions = model.predict(test_x)

output = pd.DataFrame({'Id':test_x0.Id , 'SalePrice':predictions})

output.to_csv('submission.csv',index = False)